In [15]:
from functions.data_by_country import data_by_country
from utils.data_loader import load_data_parquet
import matplotlib.pyplot as plt
import pandas as pd
import pycountry
import plotly.graph_objects as go

In [16]:
df = load_data_parquet()
df.dtypes

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
City                                   category
Country                                category
Latitude                                 object
Longitude                                object
Country_ISO                            category
dtype: object

In [17]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,Country_ISO
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E,DNK
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E,DNK
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E,DNK
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E,DNK
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E,DNK


In [18]:
df = data_by_country(df)
df = df.dropna()

#df = df.pivot(index="dt", columns="Country",
#              values='AverageTemperature').dropna()

# %%
# data = df.loc["2013-01-01"] - df.loc["1889-01-01"]
# data = data.sort_values()

c:\Users\gaute\OneDrive\Malmø\1st semester\AI\World_Temperature_Analysis\functions\data_by_country.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [19]:
df.head()

,Country,Country_ISO,dt,AverageTemperature
1070,Afghanistan,AFG,1833-01-01,1.1450
1071,Afghanistan,AFG,1833-02-01,2.1684
1072,Afghanistan,AFG,1833-03-01,6.5162
1073,Afghanistan,AFG,1833-04-01,12.8322
1074,Afghanistan,AFG,1833-05-01,18.5448


In [20]:
# Data per year

df2 = df.groupby(df['dt'].dt.year)

In [21]:
df2

In [22]:
new_df = []

for i in df2:
    print(i)
    new_df.append(i)
    
# new_df = pd.DataFrame(
#     new_df
# )

(1743,                          Country Country_ISO         dt  AverageTemperature
518240                   Albania         ALB 1743-11-01           12.686000
3627680                  Austria         AUT 1743-11-01            2.607800
6218880                  Belarus         BLR 1743-11-01            1.128875
6737120                  Belgium         BEL 1743-11-01            7.363571
8291840   Bosnia and Herzegovina         BIH 1743-11-01            6.615800
9846560                 Bulgaria         BGR 1743-11-01            6.456714
12956000                  Canada         CAN 1743-11-01           -0.206687
17620160                 Croatia         HRV 1743-11-01            6.622333
19174880                 Czechia         CZE 1743-11-01            4.530200
20211360                 Denmark         DNK 1743-11-01            6.221000
24357280                 Estonia         EST 1743-11-01            0.269500
25393760                 Finland         FIN 1743-11-01            0.619000
25912

In [23]:
df = df[df["dt"]=="1950"]

In [25]:

# Different colors https://plotly.com/python/builtin-colorscales/



fig = go.Figure(data=go.Choropleth(
    locations = df['Country_ISO'],
    z = df['AverageTemperature'],
    text = df['Country'],
    colorscale = 'RdBu',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = 'Celsius',
    colorbar_title = 'Average Temperatures',
    
))

# fig.update_layout(
#     title_text='2014 Global GDP',
#     geo=dict(
#         showframe=False,
#         showcoastlines=False,
#         projection_type='equirectangular'
#     ),
#     annotations = [dict(
#         x=0.55,
#         y=0.1,
#         xref='paper',
#         yref='paper',
#         text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
#             CIA World Factbook</a>',
#         showarrow = False
#     )]
# )

fig.show()